<a href="https://colab.research.google.com/github/Rinniedh/Python_practice/blob/main/Diane_Hoang_Lab_Assignment_03_Spoken_Language_Processing_Using_Cognitive_Services_and_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab Assignment 03 - Spoken Language Processing Using Cognitive Services and Python

In this lab assignment, we will be using **Microsoft Azure Cognitive Services** to build a system that can convert spoken language to text, and will learn to analyze the performance of the speech recognition tool.

By the time you have completed this lab, you will have achieved all of the following learning objectives:

## Learning Objectives

* Have the ability to configure and provision the *Speech* cognitive service as a resource on the Microsoft Azure platform.
* Use Python to connect to and interact with the cognitive service described above via its API.
* Have the ability to iteratively send audio recordings to the Speech cognitive service, and process the results.
* Analyze the results obtained from the Speech cognitive service and assess the quality of those results.
* Continue to develop skills working with and analyzing data in Python.

## About the Speech Cognitive Service

According to Microsoft:

> "The Speech service is the unification of speech-to-text, text-to-speech, and speech-translation into a single Azure subscription."

The Speech service thus contains tools that can convert spoken language into written text, can convert written text into spoken language, and can translate spoken language into other languages in either written or spoken form.

You may learn more about the Speech service [on this website](https://docs.microsoft.com/en-us/azure/cognitive-services/Speech-Service).

## Lab Prerequisites

To complete this lab, you will need to complete the following tasks:

* Sign into your Microsoft Azure account.
*   Add the "Speech" resource to your Azure account. To do this, follow these steps:

  1. After signing into your Azure account, use the search box to search for "Cognitive Services".
  2. Click the "Add" button.
  3. Next to the list of cognitive services, click the "See More" link.
  4. Select the "Speech" option.
  5. Click the "Create" button.
  6. Provide a unique name for the resource.
  7. Be sure that your "Azure for Students" subscription is selected, and that you have chosen a location for the resource (e.g., "(US) West US 2").
  8. Choose the "F0" (free) pricing tier. This tier allows one concurrent call to the API, with a maximum of 5 hours of spoken audio translated into written text per month.
  9. Choose or create a resource group for the Speech resource.
  10. Wait for your new resource to be deployed (this may take a minute or two).
  11. Go to your new Speech resource.
  12. **Copy and paste your key and location into the code cell below**. You will need these values in order to be able to make calls to the API from your Python code.
  13. Be sure to run the code cell below before continuing with the lab assignment.

In [1]:
#copy and paste your Speech API key and location from Azure into the variables below:
speech_api_key = '2b0b4d4568244621ad8e8aac6eb76d66'
location = 'westus2'

## Install Azure Speech and Import Libraries

The Azure Speech service is not installed by default in most Jupyter Notebook environments, so we'll need to install it. As usual, we'll also need to import all of the libraries that we'll use in the lab assignment.

Run the code cell below to install the speech and language understanding services, and import all of the necessary libraries.

In [2]:
#import libraries
import IPython
import json
import pandas as pd
import requests
import time
from IPython.display import clear_output
from scipy import stats
from string import punctuation

## Load Speech Data

The data for this lab assignment are recordings (in *WAV* format) of people who grew up in different regions of the United States speaking a variety of sentences in English. There are two parts to the data:

1. The recordings (i.e., the *WAV* files), and
2. A CSV file that contains information about the speakers and the sentences that are spoken in the *WAV* files.

Before continuing, be sure that you have uploaded all of the files that you will need for this assignment. In total, there are **252** data files, consisting of 1 CSV file and 251 WAV files.

**TIP**: Google Colab supports multiple simultaneous file uploads, so rather than uploading each data file individually, simply select all of the files, and upload them at the same time.

### Dataset Variables

The CSV file for the first part of this lab assignment contains the following variables:
* <u>id</u>: A unique identifier for each speech sample
* <u>dialect_region</u>: The region of the United States where the speaker grew up. The dialect regions are:
  * **DR1**:  New England
  * **DR2**:  Northern
  * **DR3**:  North Midland
  * **DR4**:  South Midland
  * **DR5**:  Southern
  * **DR6**:  New York City
  * **DR7**:  Western
  * **DR8**:  Multiple Regions (the speaker lived in several different regions while growing up)
* <u>speaker_id</u>: A unique identifier for each speaker
* <u>speaker_gender</u>: The gender of the speaker
* <u>sentence_code</u>: A unique identifier for the sentence being spoken by the speaker
* <u>filename</u>: The name of the WAV file that contains the recording of the speaker saying the current sentence
* <u>sentence</u>: A text version of what the speaker is saying in the associated WAV file

Run the code cell below to load the CSV file into a pandas dataframe, and preview the first few rows of data.

In [3]:
#load the dataset into a pandas dataframe
df = pd.read_csv('Lab Assignment 03 - Data.csv', index_col='id')

#show the first few rows of data
df.head(10)

,dialect_region,speaker_id,speaker_gender,sentence_code,filename,sentence
id,,,,,,
1,DR7,MPAR0,Male,SX136,MPAR0_SX136.wav,Scientific progress comes from the development...
2,DR7,MDLR0,Male,SX63,MDLR0_SX63.wav,Continental drift is a geological theory.
3,DR2,MRFK0,Male,SA1,MRFK0_SA1.wav,She had your dark suit in greasy wash water al...
4,DR7,MHXL0,Male,SA2,MHXL0_SA2.wav,Don't ask me to carry an oily rag like that.
5,DR7,MKAG0,Male,SX439,MKAG0_SX439.wav,That stinging vapor was caused by chloride vap...
6,DR4,MJLS0,Male,SI1726,MJLS0_SI1726.wav,And men also used vacuum cleaners in both room...
7,DR4,MMDM0,Male,SX51,MMDM0_SX51.wav,Ambidextrous pickpockets accomplish more.
8,DR3,MMEB0,Male,SI1357,MMEB0_SI1357.wav,The population can thereby replenish itself an...
9,DR5,FTLG0,Female,SA2,FTLG0_SA2.wav,Don't ask me to carry an oily rag like that.


Run the code cell below to replace all of the dialect region codes with their textual labels. This will make interpreting your results much easier later on.

In [4]:
#recode dialect regions
dialect_regions = {'DR1': 'New England', 'DR2': 'Northern', 'DR3': 'North Midland', 'DR4': 'South Midland', 'DR5': 'Southern', 'DR6': 'New York City', 'DR7': 'Western', 'DR8': 'Multiple Regions'}
df['dialect_region'] = [dialect_regions[dialect_region] for dialect_region in df.dialect_region]

#display number of rows for each recoded dialect region
df.dialect_region.value_counts()

dialect_region
Northern            45
Western             43
North Midland       43
South Midland       40
Southern            30
New York City       20
New England         17
Multiple Regions    12
Name: count, dtype: int64

## Descriptive Statistics

It's always a good idea to familiarize yourself with your data at the beginning of a new project, so let's take a look at a few descriptive statistics...

#### Task 01:
Write a line of code in the cell below that will display the number of sentences in the "Speech" dataset that were spoken by female speakers.

#### Question 01:

How many sentences in the "Speech" dataset were spoken by female speakers?

In [6]:
#display the number of sentences in the "Speech" dataset that were spoken by female speakers

sentence_total_female = df[df['speaker_gender'] == 'Female'].shape[0]

print("Number of sentences spoken by female speakers:", sentence_total_female)

Number of sentences spoken by female speakers: 78


#### Task 02:
Several of the sentences in the "Speech" dataset were spoken by multiple speakers. For example, the sentence with `sentence_code = 'SA1'` was spoken by 33 different speakers in the dataset. Write a line of code in the cell below that will display the total number of unique sentences in the dataset.

#### Question 02:
How many unique sentences are in the "Speech" dataset?

In [8]:
#display the total number of unique sentences in the "Speech" dataset

total_unique_sentences = df['sentence_code'].nunique()
print("Total number of unique sentences in the dataset:", total_unique_sentences)

Total number of unique sentences in the dataset: 169


#### Task 03:
Write a line of code in the cell below to generate a cross-tabulation table that will show the number of sentences spoken by male and female speakers in each dialect region.

#### Question 03:
How many sentences in the "Speech" dataset were spoken by male speakers with a *New York City* dialect?

In [9]:
#create a cross-tabulation table that contrasts speaker gender with dialect region

cross_tab = pd.crosstab(df['dialect_region'], df['speaker_gender'])
print(cross_tab)

# 9

speaker_gender    Female  Male
dialect_region                
Multiple Regions       3     9
New England            6    11
New York City         11     9
North Midland         12    31
Northern              17    28
South Midland          9    31
Southern              12    18
Western                8    35


## Length of Sentences
Estimates vary, but an average person speaking in English can be expected to speak between 100 and 150 words every 60 seconds. This is important because the Azure "Speech" cognitive service that we'll be using in this lab assignment accepts a maximum of 15 seconds of audio per transaction. Based on the speaking rate described above, this means that the maximum number of words that any sentence in our dataset can contain is between 25 and 37.5 words. To be conservative, we should use the *lower bound* of this range. We thus need to determine whether any of the sentences in our "Speech" dataset contain more than <u>25 words</u>.

#### Task 04:
Write some code in the cell below that will determine the maximum number of ***words*** contained in any of the sentences in the "Speech" dataset.

**TIP**: One strategy for determining the number of words in a sentence is to split the sentence on the whitespace (' ') character.

#### Question 04:
Among all of the sentences in the "Speech" dataset, what is the maximum number of words appearing in any sentence?

In [10]:
#determine the maximum number of words appearing in any of the sentences in the "Speech" dataset

max_words = max(len(sentence.split()) for sentence in df['sentence'])
print("Maximum number of words in any sentence:", max_words)


Maximum number of words in any sentence: 18


## Ensure that You Can Connect to the Azure Speech Cognitive Service

Now that we're more familiar with our dataset, let's connect to the Speech cognitive service. As noted at the beginning of the lab, you will need your API key and location in order to connect to the Speech API, so please be sure that you followed the earlier instructions in this assignment, and added your key and endpoint values to the appropriate variables.

When you're certain that you've stored your key and endpoint values in the `speech_api_key` and `location` variables, proceed to the next section.



## The `recognize_speech()` Function
Next, let's create a function named `recognize_speech` that will send an audio recording to the Azure Speech REST API, and return what the speaker said in text format. This function will take the name of the audio recording file as an input parameter, and will return *three* results:
1. A Boolean value (True/False) indicating whether the speech was recognized successfully;
2. A string containing the text of what the speaker said (if the speech recognition succeeded); and
3. A string containing a description of the error (if the speech recognition failed).

Run the code cell below to make this function available to your project.

In [11]:
#define a function that will send audio files to the Speech API, and return the results
def recognize_speech(filename):
  #define the URL for the REST API request
  request_url = 'https://' + location + '.stt.speech.microsoft.com/speech/recognition/conversation/cognitiveservices/v1'

  #define the headers and the query parameters for the REST API request
  headers = {'Ocp-Apim-Subscription-Key': speech_api_key, 'Content-type': 'audio/wav; codecs=audio/pcm; samplerate=16000', 'Accept': 'application/json'}
  params = {'language': 'en-US'}

  #send the audio file to the REST API, and get the results
  with open(filename, 'rb') as payload:
    response = requests.request('POST', request_url, headers=headers, params=params, data=payload)
    results = json.loads(response.text)

    #sleep for 1.1 seconds (in deference to the API's limit of one transaction per second)
    time.sleep(1.1)

    if results['RecognitionStatus'] == 'Success':
      return True, results['DisplayText'], ''
    else:
      error_message = 'An error occurred. Details: ' + results
      return False, '', error_message

## Test the Speech Recognition Tool
Now that you've successfully connected to the Azure Speech cognitive service, let's test its speech recognition capabilities to verify that we're able to send data to the API and receive a response.

If you have audio capabilities on your computer, you may run the code cell below in order to listen to the sample audio recording that will be used to test the API.

In [12]:
#play the sample audio recording that will be used to test the API
IPython.display.Audio('ApiTest.wav', autoplay=True)

In the sample audio recording, Snoop Dogg says, "I was late 'cause I went to go get some chicken wings."

Run the code cell below to send the sample audio recording to the API. If everything is working properly, the Speech service should recognize what Snoop Dogg is saying, and should return a textual sentence that matches the spoken sentence in the recording.

In [13]:
#test the speech recognizer
success, recognized_speech, error_message = recognize_speech('ApiTest.wav')

#display the results of the test
if success:
  print('The following speech was recognized: "{}"'.format(recognized_speech))
else:
  print(error_message)

The following speech was recognized: "I was late because I went to go get some chicken wings."


## Process Audio Recordings

If you've reached this point in the lab, then you've already been able to set up a Speech service on the Microsoft Azure platform and have been able to successfully communicate with the service's REST API using Python. Congratulations!

Our next task will be to process all of the audio recordings in our "Speech" dataset. This will involve the following steps for each recording:

1. Send the audio recording file to the API, and get the resulting recognized speech (in text format).
2. Store the recognized speech in a list.

Once these steps have been completed for each audio recording, we will add a new column to our dataframe that holds the text of the recognized speech for each recording, and will add another new column to the dataframe that indicates whether the results returned by the API were an exact match with what each speaker actually said.

#### Task 05:
Write a `for` loop in the code cell below that will iterate through all of the records in the dataframe, and send the `filename` for each audio recording to the API for processing. For each result, append the recognized speech to the provided list (if the API call was successful), or append an empty string ('') to the provided list (if the API call failed).

***Notes***:
1. There should not be any errors (i.e., all audio samples should be able to be processed by the API successfully).
2. It may take several minutes for your loop to run, given that there are 250 audio recordings in the dataset!

In [21]:
#Define a list to hold the text of the recognized sentences.
recognized_sentences = []

#write a 'for' loop to send each audio recording file in the dataframe to the API
current_recording = 0
total_recordings = df.shape[0] #the number of rows in the dataframe

#WRITE YOUR 'FOR' LOOP HERE - be sure to use 'filename' as the name of the loop's control variable
for filename in df['filename']:

  #display overall progress
  clear_output(wait=True)
  current_recording += 1
  print('Processing audio recording {} of {}...'.format(current_recording, total_recordings))

  #send the current audio recording to the API, and get the results
  success, recognized_speech, error_message = recognize_speech(filename)

  #If the speech was recognized successfully, add the recognized speech to the 'recognized_sentences' list.
  #Otherwise, add an empty string ('') to the list.
  recognized_sentences.append(recognized_speech if success else '')

#add a column to the dataframe that contains the recognized sentences
df['api_sentence'] = [api_sentence for api_sentence in recognized_sentences]

#add a column to the dataframe that indicates whether the speech recognized by the
#API is an exact match with what the speaker actually said
df['exact_match'] = df['sentence'] == df['api_sentence']

#update status
clear_output(wait=True)
print('Processing complete!')

Processing complete!


Now that all of the audio recordings have been processed, write a line of code in the cell below that will display the number of sentences that the API recognized with perfect accuracy.

#### Question 05:
How many of the audio recordings did the API recognize with 100% accuracy?

In [22]:
#display the number of audio recordings that the API recognized with 100% accuracy
num_exact_matches = df['exact_match'].sum()
print('Number of sentences recognized with perfect accuracy:', num_exact_matches)

Number of sentences recognized with perfect accuracy: 154


## Punctuation and Capitalization
The performance of the Azure Speech service currently appears to be rather mediocre, but this may be due to problems with puctuation and capitalization. Put differently, if we compare the text of what the speaker actually said with the text of what the Speech service thought that the speaker said *after accounting for differences in punctuation and capitalization*, then the performance of the API may seem better.

#### Task 06:
Run the code cell below to add a new column named `fuzzy_match` to your dataframe that indicates whether what the speaker actually said matches what the Speech API thought the speaker said after accounting for differences in punctuation and capitalization.

In [23]:
#create a list to hold whether each sentence matches each API-inferred sentence,
#after accounting for differences in punctuation and capitalization
fuzzy_matches = []

#iterate through the dataframe
for sentence, api_sentence in df[['sentence', 'api_sentence']].itertuples(index=False):
  #remove punctuation and convert sentences to lower case
  sentence = ''.join(c for c in sentence if c not in punctuation).lower()
  api_sentence = ''.join(c for c in api_sentence if c not in punctuation).lower()
  #record whether the sentences match after accounting for differences in punctuation and capitalization
  fuzzy_matches.append(sentence == api_sentence)

#add a 'fuzzy_match' column to the dataframe
df['fuzzy_match'] = fuzzy_matches

Write a line of code in the cell below that will display the number of sentences that the API recognized with perfect accuracy, after accounting for differences in punctuation and capitalization.

#### Question 06:
How many of the audio recordings did the API recognize accurately after accounting for differences in punctuation and capitalization?

In [24]:
#display the number of audio recordings that the API recognized accurately,
#after accounting for differences in punctuation and capitalization.

# Remove punctuation and capitalize both the recognized and actual sentences before comparing them.
df['api_sentence_no_punct'] = df['api_sentence'].str.replace('[.,!?\s]+', '').str.capitalize()
df['sentence_no_punct'] = df['sentence'].str.replace('[.,!?\s]+', '').str.capitalize()

# Calculate the number of sentences that match after removing punctuation and capitalizing.
num_matches_no_punct = (df['api_sentence_no_punct'] == df['sentence_no_punct']).sum()

# Print the results.
print('Number of sentences recognized with perfect accuracy (ignoring punctuation and capitalization):', num_matches_no_punct)

Number of sentences recognized with perfect accuracy (ignoring punctuation and capitalization): 191


## Speech Recognition Accuracy by Gender and Dialect
After computing our `fuzzy_match` column, we now have a better idea of how the Speech service actually performs. Next, let's investigate the sources of the API's mistakes in more detail by evaluating how the speaker's gender and dialect region affect the API's speech recognition performance.

#### Task 07:
Write some code in the cell below that will display how well the Speech service performs based on the speaker's gender. Be sure to use the `fuzzy_match` column as the basis for your accuracy calculations. Also, be sure to account for differences in the number of male vs. female speakers (e.g., by normalizing your calculations into percentages).

#### Question 07:
After accounting for differences in punctuation and capitalization, does the Speech API appear to be better at recognizing male speakers or female speakers?

In [25]:
#Display how well the Speech service performs based on the speaker's gender.
#Be sure to use the `fuzzy_match` column as the basis for your accuracy calculations.

# Group the data by speaker gender.
grouped_df = df.groupby('speaker_gender')

# Calculate the number of fuzzy matches for each gender.
num_fuzzy_matches = grouped_df['fuzzy_match'].sum()

# Calculate the total number of recordings for each gender.
num_recordings = grouped_df['filename'].count()

# Calculate the percentage of fuzzy matches for each gender.
fuzzy_match_percentages = (num_fuzzy_matches / num_recordings) * 100

# Print the results.
print('Speech recognition accuracy by speaker gender:')
for gender, percentage in fuzzy_match_percentages.items():
    print(f'- {gender}: {percentage:.2f}%')

Speech recognition accuracy by speaker gender:
- Female: 87.18%
- Male: 89.53%


#### Task 08:
Write some code in the cell below that will display how well the Speech service performs based on the speaker's dialect region. As with the previous task, be sure to use the `fuzzy_match` column as the basis for your accuracy calculations, and also be sure to account for differences in the number of speakers from each dialect region (e.g., by normalizing your calculations into percentages).

#### Question 08:
After accounting for differences in punctuation and capitalization, for which dialect region is the Speech API the most and least accurate?

In [26]:
#Display how well the Speech service performs based on the speaker's dialect region.
#Be sure to use the `fuzzy_match` column as the basis for your accuracy calculations.

# Group the data by speaker dialect region.
grouped_df = df.groupby('dialect_region')

# Calculate the number of fuzzy matches for each dialect region.
num_fuzzy_matches = grouped_df['fuzzy_match'].sum()

# Calculate the total number of recordings for each dialect region.
num_recordings = grouped_df['filename'].count()

# Calculate the percentage of fuzzy matches for each dialect region.
fuzzy_match_percentages = (num_fuzzy_matches / num_recordings) * 100

# Print the results.
print('Speech recognition accuracy by speaker dialect region:')
for region, percentage in fuzzy_match_percentages.items():
    print(f'- {region}: {percentage:.2f}%')


Speech recognition accuracy by speaker dialect region:
- Multiple Regions: 75.00%
- New England: 88.24%
- New York City: 95.00%
- North Midland: 88.37%
- Northern: 91.11%
- South Midland: 82.50%
- Southern: 93.33%
- Western: 90.70%


## Are Gender-Based Differences Statistically Significant?
The analysis above showed that the Speech Service appears to perform differently depending on the speaker's gender, but is this difference statistically significant? Put differently, do we have enough evidence to conclude that the Speech API *generally* performs better across *all* English-speaking people in the United States when recognizing speakers of one gender vs. another?

To answer this question, we can perform a **Welch's t-Test** to compare the API's accuracy when recognizing the speech of male vs. female speakers. Welch's t-test is a good choice in this situation because it allows for the two samples that are being compared (i.e., the results for male vs. female speakers) to have different sample sizes and unequal variances.

#### Task 09:
Write some code in the cell below that will perform a two-tailed Welch's t-test that compares the API's performance in accurately recognizing male vs. female speakers. Be sure to use the `fuzzy_match` column in the dataset as the basis for determining if the API was accurate or not.
**TIP**: Welch's t-test is implemented in the `ttest_ind()` function in the scipy `stats` library. This library has already been imported into this notebook for you. Be sure to set the `equal_var` parameter in the `ttest_ind()` function to `False` in order to ensure that a Welch t-test is performed.

#### Question 09:
To what extent is the difference in the Speech API's performance with male vs. female speakers statistically significant?

In [27]:
#perform a Welch's t-test to determine if the API's performance with male vs. female
#speakers is statistically significant.

import scipy.stats as stats

# Extract the fuzzy_match values for male and female speakers.
male_fuzzy_matches = df[df['speaker_gender'] == 'male']['fuzzy_match']
female_fuzzy_matches = df[df['speaker_gender'] == 'female']['fuzzy_match']

# Perform Welch's t-test to compare the accuracy for male and female speakers.
t_statistic, p_value = stats.ttest_ind(male_fuzzy_matches, female_fuzzy_matches, equal_var=False)

# Print the results.
print('Welch\'s t-test results:')
print(f'- t-statistic: {t_statistic:.4f}')
print(f'- p-value: {p_value:.4f}')

# Interpret the results.
if p_value < 0.05:
    print('There is a statistically significant difference in the API\'s accuracy for male and female speakers.')
else:
    print('There is no statistically significant difference in the API\'s accuracy for male and female speakers.')

Welch's t-test results:
- t-statistic: nan
- p-value: nan
There is no statistically significant difference in the API's accuracy for male and female speakers.


## On the Nature of Error
We now know that the Azure Speech service occasionally makes mistakes when performing speech recognition, and that the frequency of these mistakes varies according to the speaker's gender and dialect region. But *error* is a subtle concept, particularly in the context of spoken language. Perhaps we shouldn't judge the Speech API's performance based only on whether it made a mistake, but instead should consider the severity of its mistakes.

#### Task 10:
For cases in which the Speech API made a mistake, write some code in the cell below that will allow you to visually compare what the speakers in the dataset actually said with what the Speech API thought they said.

#### Question 10:
How would you characterize the severity of the mistakes that the Speech API makes?

In [28]:
#visually compare what the speakers actually said vs. what the Speech API thought they said
# Create a new DataFrame to hold the mismatched sentences.
mismatched_sentences_df = df[df['fuzzy_match'] == 0]

# Display the actual and recognized sentences side-by-side.
mismatched_sentences_df[['sentence', 'api_sentence']]

,sentence,api_sentence
id,,
12,The haunted house was a hit due to outstanding...,The Haunted House was a hit due to outstanding...
19,The trouble is that like many symbols it doesn...,"The trouble is that like many symbols, it does..."
22,"The fifth jar contains big, juicy peaches.",The 5th jar contains big juicy Peaches.
30,Shipbuilding is a most fascinating process.,Shipbuilding is of most fascinating process.
35,She had your dark suit in greasy wash water al...,She had a dock suit in greasy wash water all y...
43,The thick elm forest was nearly overwhelmed by...,The thick Air Force was nearly overwhelmed by ...
50,Shell shock caused by shrapnel is sometimes cu...,Show shock caused by shrapnel is sometimes cur...
55,He liked to nip ear lobes of unsuspecting visi...,He liked to nip earlobes of unsuspecting visit...
73,Development requires a long-term approach.,Development requires a long term approach.


##End of Lab Assignment 03!